In [1]:
import os
import keras
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook

Using TensorFlow backend.


In [100]:
########################
##    Reading data    ##
########################

matrices_0 = np.load('matrix_data/matrices_0.npy')
matrices_1 = np.load('matrix_data/matrices_1.npy')
matrices_2 = np.load('matrix_data/matrices_2.npy')
matrices_3 = np.load('matrix_data/matrices_3.npy')

print('matrices_0 :', matrices_0.shape)
print('matrices_0 :', matrices_1.shape)
print('matrices_0 :', matrices_2.shape)
print('matrices_0 :', matrices_3.shape, end='\n\n')

data = np.row_stack((matrices_0,
                     matrices_1,
                     matrices_2,
                     matrices_3))

print('Final data :', data.shape)

matrices_0 : (2070, 68, 53)
matrices_0 : (2070, 68, 53)
matrices_0 : (2070, 68, 53)
matrices_0 : (2070, 68, 53)

Final data : (8280, 68, 53)


In [68]:
############################
##    Days range input    ##
############################

start = np.zeros(data[0].shape)
days_input = np.array([start+i for i in range(data.shape[0])]).astype(int)
days_input %= 31
days_input = days_input / np.max(days_input)
print('days_input :', days_input.shape)

days_input : (8280, 68, 53)


In [74]:
##############################
##    Months range input    ##
##############################

start = np.zeros(data[0].shape)
months_input = np.array([start+i for i in range(data.shape[0])]).astype(int)
months_input //= 31
months_input %= 12
months_input = months_input / np.max(months_input)
print('months_input :', months_input.shape)

months_input : (8280, 68, 53)


In [101]:
print("""
????   ?????  ?????   ???   ????   ?????
?   ?  ?      ?      ?   ?  ?   ?  ?
????   ????   ????   ?   ?  ????   ????
?   ?  ?      ?      ?   ?  ?  ?   ?
?   ?  ?      ?      ?   ?  ?   ?  ?
????   ?????  ?       ???   ?   ?  ????? 
""")
print('data :', data.shape)
#############################################
input_data = np.stack((data, months_input), axis=3)
target_data = data
#############################################
print("""

 ???   ?????  ?????  ?????  ????   
?   ?  ?        ?    ?      ?   ?  
?????  ????     ?    ????   ????
?   ?  ?        ?    ?      ?  ?
?   ?  ?        ?    ?      ?   ?
?   ?  ?        ?    ?????  ?   ?
""")
print('input_data   :', input_data.shape)
print('target_data  :', target_data.shape)


????   ?????  ?????   ???   ????   ?????
?   ?  ?      ?      ?   ?  ?   ?  ?
????   ????   ????   ?   ?  ????   ????
?   ?  ?      ?      ?   ?  ?  ?   ?
?   ?  ?      ?      ?   ?  ?   ?  ?
????   ?????  ?       ???   ?   ?  ????? 

data : (8280, 68, 53)


 ???   ?????  ?????  ?????  ????   
?   ?  ?        ?    ?      ?   ?  
?????  ????     ?    ????   ????
?   ?  ?        ?    ?      ?  ?
?   ?  ?        ?    ?      ?   ?
?   ?  ?        ?    ?????  ?   ?

input_data   : (8280, 68, 53, 2)
target_data  : (8280, 68, 53)


In [109]:
############################
##    Train-Test Split    ##
############################

train_input = input_data[:9*input_data.shape[0]//10][:-31]
test_input = input_data[9*input_data.shape[0]//10:][:-31]

train_target = np.roll(target_data[:9*target_data.shape[0]//10],-31,axis=0)[:-31]
test_target = np.roll(target_data[9*target_data.shape[0]//10:],-31,axis=0)[:-31]

np.save('train_input.npy', train_input)
np.save('train_target.npy', train_target)
#########################################
np.save('test_input.npy', test_input)
np.save('test_target.npy', test_target)

print('train_input  :', train_input.shape)
print('train_target :', train_target.shape, end='\n\n')
print('test_input   :', test_input.shape)
print('test_target  :', test_target.shape)

train_input  : (7421, 68, 53, 2)
train_target : (7421, 68, 53)

test_input   : (797, 68, 53, 2)
test_target  : (797, 68, 53)


In [116]:
#################
##    Model    ##
#################

input_matrix = keras.layers.Input(shape=train_input[0].shape)

logit = keras.layers.Flatten()(input_matrix)
logit = keras.layers.Dense(128, activation='tanh')(logit)
logit = keras.layers.Dense(128, activation='tanh')(logit)
logit = keras.layers.Dense(3604, activation='linear')(logit)
logit = keras.layers.Activation('relu')(logit)
logit = keras.layers.Reshape(train_target[0].shape)(logit)

model = keras.models.Model(inputs=input_matrix, outputs=logit)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 68, 53, 2)         0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 7208)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               922752    
_________________________________________________________________
dense_20 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_21 (Dense)             (None, 3604)              464916    
_________________________________________________________________
activation_7 (Activation)    (None, 3604)              0         
_________________________________________________________________
reshape_7 (Reshape)          (None, 68, 53)            0         
Total para

In [117]:
##########################################
##   Compiling the model with L1 loss   ##
##########################################

model.compile(loss='mae', optimizer=keras.optimizers.Adam(lr=0.001))

In [118]:
############################
##   Training the model   ##
############################

model.fit(x=train_input[:1], y=train_target[:1],
          epochs=1000,
          batch_size=32,
          validation_data=(test_input, test_target))

Train on 1 samples, validate on 797 samples
Epoch 1/10
1/1 [==============================] - 1s 592ms/step - loss: 1.2267 - val_loss: 1.3601
Epoch 2/10
1/1 [==============================] - 0s 166ms/step - loss: 1.2221 - val_loss: 1.3589
Epoch 3/10
1/1 [==============================] - 0s 160ms/step - loss: 1.2029 - val_loss: 1.3566
Epoch 4/10
1/1 [==============================] - 0s 166ms/step - loss: 1.1853 - val_loss: 1.3540
Epoch 5/10
1/1 [==============================] - 0s 166ms/step - loss: 1.1722 - val_loss: 1.3509
Epoch 6/10
1/1 [==============================] - 0s 189ms/step - loss: 1.1633 - val_loss: 1.3477
Epoch 7/10
1/1 [==============================] - 0s 181ms/step - loss: 1.1581 - val_loss: 1.3445
Epoch 8/10
1/1 [==============================] - 0s 153ms/step - loss: 1.1532 - val_loss: 1.3415
Epoch 9/10
1/1 [==============================] - 0s 149ms/step - loss: 1.1491 - val_loss: 1.3387
Epoch 10/10
1/1 [==============================] - 0s 167ms/step - loss: 1

In [115]:
###################################
##   Saving the resulted model   ##
###################################

model.save('model_1.h5')